1. Load data and model

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Check your setup.")


GPU is available!
Using GPU: NVIDIA GeForce RTX 4080 Laptop GPU


In [3]:
# Import required libraries
import os
from datasets import Dataset

# Function to read IOB files and split into paragraphs
def read_iob_file(file_path):
    """Read IOB file and convert to token-label samples"""
    examples = []
    words, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.split("\t")
                words.append(token)
                labels.append(label.strip())
            else:
                # End of a sample
                if words and labels:
                    examples.append({"tokens": words, "labels": labels})
                    words, labels = [], []
    # Add the last example if file doesn't end with a blank line
    if words and labels:
        examples.append({"tokens": words, "labels": labels})
    return examples



In [4]:
# Load datasets using the updated function
train_path = 'C:\\S24-25\\TxM\\dataset\\train.tsv'
val_path = 'C:\\S24-25\\TxM\\dataset\\val_gold.tsv'
test_path = 'C:\\S24-25\\TxM\\dataset\\test_gold.tsv'
train_data = read_iob_file(train_path)
val_data = read_iob_file(val_path)
test_data = read_iob_file(test_path)

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}, Test samples: {len(test_data)}")


Train samples: 998, Val samples: 124, Test samples: 126


In [6]:
from transformers import AutoTokenizer, AutoModelForPreTraining
model_checkpoint="kamalkraj/bioelectra-base-discriminator-pubmed"
tokenizer = AutoTokenizer.from_pretrained("kamalkraj/bioelectra-base-discriminator-pubmed")
model = AutoModelForPreTraining.from_pretrained("kamalkraj/bioelectra-base-discriminator-pubmed")
# Define label mappings
label_list = ["O", "B-ADR", "I-ADR", "B-DRU", "I-DRU", "B-DIS", "I-DIS", "B-SYM", "I-SYM"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        is_split_into_words=True,
        padding="max_length",      # or True, depending on your preference
        truncation=True,          # enable truncation
        max_length=512,           # explicitly set maximum sequence length 
        return_overflowing_tokens=False,
    )
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore padding
            elif word_idx != previous_word_idx:
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize datasets
train_dataset = Dataset.from_list(train_data).map(tokenize_and_align_labels, batched=True)
test_dataset = Dataset.from_list(test_data).map(tokenize_and_align_labels, batched=True)
val_dataset = Dataset.from_list(val_data).map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 124/124 [00:00<00:00, 2656.52 examples/s]


In [7]:
from transformers import AutoModelForTokenClassification


# Load the pre-trained BERT model

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id,
    ignore_mismatched_sizes=True
)


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at kamalkraj/bioelectra-base-discriminator-pubmed and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. Train and evaluate with default parameters on the test set

In [8]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./bio_electra_logs",
    logging_steps=10,
)


In [9]:
from transformers import Trainer
from evaluate import load
import torch

# Load metric for evaluation
metric = load("seqeval")

def compute_metrics(predictions):
    predictions, labels = predictions
    predictions = torch.argmax(torch.tensor(predictions), dim=2)
    
    # Convert predictions and labels to CPU and then to plain Python integers
    predictions = predictions.cpu().numpy()
    
    true_labels = [
        [id_to_label[label] for label in label_seq if label != -100] 
        for label_seq in labels
    ]
    true_predictions = [
        [id_to_label[int(p)] for (p, l) in zip(prediction, label_seq) if l != -100]  # Convert tensor to int
        for prediction, label_seq in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


  5%|▌         | 10/189 [02:27<43:57, 14.73s/it]

{'loss': 1.6947, 'grad_norm': 8.60431957244873, 'learning_rate': 1.8941798941798943e-05, 'epoch': 0.16}


 11%|█         | 20/189 [04:54<40:57, 14.54s/it]

{'loss': 0.8684, 'grad_norm': 1.3569080829620361, 'learning_rate': 1.7883597883597884e-05, 'epoch': 0.32}


 16%|█▌        | 30/189 [07:18<38:08, 14.39s/it]

...

100%|██████████| 189/189 [47:15<00:00, 11.67s/it]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
100%|██████████| 189/189 [47:21<00:00, 11.67s/it]

{'eval_loss': 0.28889429569244385, 'eval_ADR': {'precision': 0.2753623188405797, 'recall': 0.3578767123287671, 'f1': 0.31124348473566643, 'number': 584}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 39}, 'eval_DRU': {'precision': 0.7798165137614679, 'recall': 0.6007067137809188, 'f1': 0.6786427145708583, 'number': 283}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 44}, 'eval_overall_precision': 0.38792221084953943, 'eval_overall_recall': 0.3989473684210526, 'eval_overall_f1': 0.39335755059678257, 'eval_overall_accuracy': 0.9083582296949906, 'eval_runtime': 4.111, 'eval_samples_per_second': 30.649, 'eval_steps_per_second': 1.946, 'epoch': 3.0}


100%|██████████| 189/189 [47:23<00:00, 15.04s/it]

{'train_runtime': 2843.3146, 'train_samples_per_second': 1.053, 'train_steps_per_second': 0.066, 'train_loss': 0.4534431260729593, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.4534431260729593, metrics={'train_runtime': 2843.3146, 'train_samples_per_second': 1.053, 'train_steps_per_second': 0.066, 'total_flos': 782372000176128.0, 'train_loss': 0.4534431260729593, 'epoch': 3.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


100%|██████████| 8/8 [00:03<00:00,  1.62it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 8/8 [00:03<00:00,  2.07it/s]

Evaluation Results: {'eval_loss': 0.28889429569244385, 'eval_ADR': {'precision': 0.2753623188405797, 'recall': 0.3578767123287671, 'f1': 0.31124348473566643, 'number': 584}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 39}, 'eval_DRU': {'precision': 0.7798165137614679, 'recall': 0.6007067137809188, 'f1': 0.6786427145708583, 'number': 283}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 44}, 'eval_overall_precision': 0.38792221084953943, 'eval_overall_recall': 0.3989473684210526, 'eval_overall_f1': 0.39335755059678257, 'eval_overall_accuracy': 0.9083582296949906, 'eval_runtime': 4.4505, 'eval_samples_per_second': 28.311, 'eval_steps_per_second': 1.798, 'epoch': 3.0}


: 

3. Train and test on val set

4. HPO